<a href="https://colab.research.google.com/github/4a6166/practice/blob/main/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# getting the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-12-05 01:46:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2023-12-05 01:46:02 (18.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
# read input in
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
len(text)

1115394

In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
# get all unique chars
chars = sorted(list(set(text)))
vocab_size= len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# char-level language model
# create mapping from char to int
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

# encoder takes a string, outputs a list of ints
encode = lambda s: [stoi[c] for c in s]

# decoder takes a list of ints, outputs a string
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
print(encode("testing encoder"))
print(decode(encode("testing encoder")))

[58, 43, 57, 58, 47, 52, 45, 1, 43, 52, 41, 53, 42, 43, 56]
testing encoder


In [ ]:
# encode input data
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
# create a test_train split

# get first 90% of data to be train data
n =  int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
# define how large you want blocks of sampled text to be
# reminder: char-level blocks, so this would equal "First Cit"

block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
# make train data arrays
X_train = train_data[:block_size]
y_train = train_data[1:block_size+1]
for t in range(block_size):
  context = X_train[:t+1]
  target = y_train[t]
  print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [ ]:
#

torch.manual_seed(1337) # setting the seed for reproducability, would want to clear the seed if going for real
batch_size = 4 # number of sequences to process in parallel
block_size = 8 # max context length for predictions

# get batches to take advantange of GPU parallel processing
def get_batch(split):
  # generate a batch of data inputs x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)

print('targets')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()}, target is {target}")

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
when input is [24], target is 43
when input is [24, 43], target is 58
when input is [24, 43, 58], target is 5
when input is [24, 43, 58, 5], target is 57
when input is [24, 43, 58, 5, 57], target is 1
when input is [24, 43, 58, 5, 57, 1], target is 46
when input is [24, 43, 58, 5, 57, 1, 46], target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43], target is 39
when input is [44], target is 53
when input is [44, 53], target is 56
when input is [44, 53, 56], target is 1
when input is [44, 53, 56, 1], target is 58
when input is [44, 53, 56, 1, 58], target is 46
when input is [44, 53, 56, 1, 58, 4

In [ ]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    #idx and targets are both (B,T) tensor of ints
    logits = self.token_embedding_table(idx) # (Batch,Time,Channel)

    if targets is None:
      loss = None
    else:
      # need to reshape logits because cross_entropy wants input that way
      B, T, C = logits.shape
      logits = logits.view(B*T, C)

      # need to reshape targets because cross_entropy wants input that way
      targets = targets.view(B*T)

      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indives in the current context
    for _ in range(max_new_tokens):
      # get predictions
      logits, loss = self(idx)
      #focus only on last time step
      logits = logits[:,-1, :] # Becomes (B, C)
      #apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B,C)
      # sample from distribution
      idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
      # append sampled index to running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)
    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.5424, grad_fn=<NllLossBackward0>)


In [ ]:
print(decode(
    m.generate(
        torch.zeros((1,1), dtype=torch.long), # creating a new line char to start off
        max_new_tokens=100)[0].tolist()
        )
    )

# outputs garbage becuase model is not trained


?jt!:BWaoUEapAIvclBYFMdmbh-?EtEZdVdfT;sdxYBKUFA.GnGacchqKBo'XoZ'ufcR'u3!O;L$.ZSS.&l$BrsRUaKev-xO3$ww


In [ ]:
# training the model

# create optimizer with Pytorch
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) #for complex models, probably use 3e-4 for the learning rate

In [ ]:
# training loop
batch_size = 32
for steps in range(10000):
  # sample a batch of data
  xb, yb = get_batch('train')

  #evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.409691095352173


In [ ]:
print(decode(
    m.generate(
        torch.zeros((1,1), dtype=torch.long), # creating a new line char to start off
        max_new_tokens=500)[0].tolist()
        )
    )



KEN:
I'stthe at te chu by. s nnk'sin, f iso t m yourouthag, CLOKEONo gelear ave d, he, n
S:
On sspoun, penoullant why icif poover-eshem aver, fr crtllins ENI binn t ge a ctio INAsth sucoks!
etrth shy'd;s rnime e aireten t

CHicof hmysofofasChe w,
LLourestousouee,
Brthe;jukne hath, be:
wibreer omaingonougio ourmend t I MI ve d the the IIOM:
S: hoy mor ine'dwak iod,
I t oure no-prt!
HAn nond umavas ithin,
fe ikiurily:
thangsea'?
O:

Y bofowars ioye t y:

Ane a thie s aghattofl thothenin:

NUCED: y


## Self-Attention Matrix Multiplication

In [ ]:
# toy example

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.random(B,T,C)
x.shape

torch.Size([4,8,2])

In [ ]:
# version 1
# we want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) #bow = bag of words
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1] # shape is (t,C)
    xbow[b,t] = torch.mean(xprev, 0)

In [ ]:
# matrix mult
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3)) # tril gives lower triangle of matrix
a = a / torch.sum(a, 1, keepdim=True) # set all 1's in triangle sum to one (2nd row should be [.5, .5, 0])
b = torch.randint(0,10,(3,2)).float() # generate random 3x2 matrix
c = a @ b # dotproduct

In [ ]:
# version 2
# back to xbow, get triangle weights

wei = torch.tril(torch.ones(T,T)) # short for weights
wei = wei /wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T,T) @ (B, T, C) => torch creates batch for wei (B,T,T) @ (B,T,C) ---> creates (B,T,C)
torch.allclose(xbow, xbow2) # True

In [ ]:
# version 2
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T)) # these tokens will start looking at each other here in other examples (57:57)
wei = wei.masked_fill(tril ==0, float('-inf')) # masked fill sets all elements where tril = 0 to -infinity
wei = F.softmax(wei, dim=-1) # softmax exponentiates each ele then divides by sum, normalizing to 1
xbow3 = wei @ x
torch.allclose(xbow, xbow3) # True